In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# https://www.datos.gov.co/resource/kwvf-nwea.geojson
# se puede descargar el archivo geojson directamente desde la url anterior

In [3]:
import geopandas as gpd

# Cargar geojson
gdf = gpd.read_file("data/coordenadas/zonificacion_aptitud_cultivo_cafe.geojson") 

# Reproyectar a WGS84, estándar para coordenadas GPS
gdf = gdf.to_crs(epsg=4326)

# Crear centroides
gdf["centroid"] = gdf.geometry.centroid
gdf["lon"] = gdf.centroid.x
gdf["lat"] = gdf.centroid.y

gdf[['lon', 'lat']].head()

,lon,lat
0,-75.286124,2.476624
1,-75.655666,6.028590
2,-75.365625,5.843742
3,-75.499060,5.776846
4,-75.432300,5.776576


In [4]:
# Limpieza y filtrado de datos
gdf.columns = [c.strip().lower() for c in gdf.columns]

# Nombres de columnas estandarizados
gdf.columns = [c.strip().lower() for c in gdf.columns]

# Disolver geometrías por municipio
gdf_diss = gdf.dissolve(
    by="cod_dane_m",
    as_index=False,
    aggfunc="first"
)

# Calcular centroides
gdf_diss["centroid"] = gdf_diss.geometry.centroid
gdf_diss["lon"] = gdf_diss.centroid.x
gdf_diss["lat"] = gdf_diss.centroid.y

gdf_diss.head()

,cod_dane_m,geometry,:id,:version,:created_at,:updated_at,municipio,departamen,cod_depart,gridcode,area_ha,aptitud,consecutiv,centroid,lon,lat
0,05001,"POLYGON ((-75.7121 6.31687, -75.71225 6.31705,...",row-6pw9.rnc7-ura8,rv-refb-z8hb~6g4j,2022-08-11 16:44:26.021000+00:00,2022-08-11 16:44:26.021000+00:00,Medellín,Antioquia,05,3,1133.15124512,Aptitud alta,255042,POINT (-75.61129 6.25853),-75.611288,6.258531
1,05002,"POLYGON ((-75.5319 5.8639, -75.5318 5.86392, -...",row-6wu4.ccw5.5wnt,rv-582d~h7u8-gcbj,2022-08-11 16:44:26.021000+00:00,2022-08-11 16:44:26.021000+00:00,Abejorral,Antioquia,05,0,13323.4296875,No apta,468,POINT (-75.43847 5.80417),-75.438474,5.804174
2,05004,"POLYGON ((-76.08065 6.75106, -76.07974 6.75107...",row-k8ns.4bai-akei,rv-egaf~76c8~3gsw,2022-08-11 16:44:26.021000+00:00,2022-08-11 16:44:26.021000+00:00,Abriaquí,Antioquia,05,8,29114.9492188,Exclusión legal,2499,POINT (-76.08573 6.62631),-76.085735,6.626307
3,05021,"POLYGON ((-75.16009 6.32325, -75.15987 6.32363...",row-mnyg_kdgp_riy6,rv-ncxt-w26e-tg6w,2022-08-11 16:44:26.021000+00:00,2022-08-11 16:44:26.021000+00:00,Alejandría,Antioquia,05,0,2455.4765625,No apta,17652,POINT (-75.09018 6.36492),-75.090177,6.364923
4,05030,"POLYGON ((-75.76273 6.01093, -75.76261 6.01108...",row-8ih3-hjdr_wsng,rv-2scz~rfj8~awxx,2022-08-11 16:44:26.021000+00:00,2022-08-11 16:44:26.021000+00:00,Amagá,Antioquia,05,8,355.02838135,Exclusión legal,241,POINT (-75.7079 6.03277),-75.707902,6.032773


In [ ]:
gdf_diss['aptitud'].unique()

array(['Exclusión legal', 'No apta', 'Aptitud alta', 'Aptitud media',
       'Aptitud baja'], dtype=object)

In [5]:
from IPython.display import HTML, display

display(HTML('<img src="data/coordenadas/categorias_zonificacion.png" width="500" height="400">'))


In [6]:
dept_relacion = [gdf_diss[["cod_depart", "departamen"]].drop_duplicates()
                  .sort_values("departamen").reset_index(drop=True)]
print(dept_relacion)


[   cod_depart                               departamen
0          91                                 Amazonas
1          05                                Antioquia
2          81                                   Arauca
3          08                                Atlántico
4          13                                  Bolívar
5          15                                   Boyacá
6          17                                   Caldas
7          18                                  Caquetá
8          85                                 Casanare
9          19                                    Cauca
10         20                                    Cesar
11         27                                    Chocó
12         25                             Cundinamarca
13         23                                  Córdoba
14         94                                  Guainía
15         95                                 Guaviare
16         41                                    Huila
17       

In [7]:
# Quindio = 63
# Cundinamarca = 25

In [8]:
# Seleccionar columnas relevantes
coordenadas = gdf_diss[
    ["cod_depart", "departamen", "cod_dane_m", "municipio", "lon", "lat", "area_ha", "aptitud"]
].copy()

# Quitar duplicados por municipio
coordenadas = coordenadas.drop_duplicates("cod_dane_m").reset_index(drop=True)

# Agregar un ID secuencial
coordenadas.insert(0, "id", range(1, 1 + len(coordenadas)))

coordenadas.head()


,id,cod_depart,departamen,cod_dane_m,municipio,lon,lat,area_ha,aptitud
0,1,05,Antioquia,05001,Medellín,-75.611288,6.258531,1133.15124512,Aptitud alta
1,2,05,Antioquia,05002,Abejorral,-75.438474,5.804174,13323.4296875,No apta
2,3,05,Antioquia,05004,Abriaquí,-76.085735,6.626307,29114.9492188,Exclusión legal
3,4,05,Antioquia,05021,Alejandría,-75.090177,6.364923,2455.4765625,No apta
4,5,05,Antioquia,05030,Amagá,-75.707902,6.032773,355.02838135,Exclusión legal


In [9]:
coordenadas.to_csv("data/coordenadas/zonificacion_coordenadas_cafe.csv", index=False)